# Loops & Orbits &mdash; Week 3 &mdash; Day 3 &mdash; Jupyter Notebook

## Ellipses from Newton's Cannon with Second-Order Runge-Kutta

Pronounce the German names well: "roon-geh koot-tuh" &mdash; not "runj kutah" which is admittedly how it looks.

**Execute the imports.** There is nothing here for you to change.

In [ ]:
# Important math functions and the constant pi:
from math import sin, cos, atan2, sqrt, pi

# We will start using the high-performance numpy library for dealing with arrays of floats:
import numpy as np

# The following import statement makes the plotting library available to us. There is also a mysterious
# statement beginning with % to work around a known Jupyter bug: https://github.com/jupyter/notebook/issues/3670
%matplotlib inline
import matplotlib.pyplot as plt

# Use unit testing framework to test each function we write:
import unittest

Next, define all the constants. *Our numbers in meters were getting really huge. All of the constants are now in kilometers instead of meters.* **Execute these assignments.**

In [ ]:
# The following two parameters are get us the strength of gravity at various heights:
g = 0.00981                    # 0.00981 km/s**2 at Earth surface
radius_of_earth = 6371.0 # Earth radius in km

# The following two variables define the cannonball launch conditions:
initial_position = [0.0, radius_of_earth + 500.0]  # cannon is on a 500 km high mountain
initial_velocity = [8.982, 0.0]                     # a good range of initial x-velocities to try is 7.6 to 9.2 km/s
# The value of 8.982 km/s was tweaked to make exactly one elliptical orbit in 200 minutes.

# The following two parameters establish the simulation time step and its maximum total duration:
delta_t = 120.0              # simulation time step in seconds
simulation_time_steps = 100  # 200 minutes worth for a delta_t of 120.0

## Functions for Working with Vectors

All of these functions you implemented previously. **Just execute this cell to get our vector functions.**

In [ ]:
# computes horizontal and vertical components of a vector and returns them as a tuple
def vector_from_length_and_angle(length: float, angle: float) -> np.ndarray:
    # we are working in degrees -- python's are expecting radians -- convert degrees to radian
    angle_in_radians = angle * pi / 180.0
    x_component = length * cos(angle_in_radians)
    y_component = length * sin(angle_in_radians)
    return np.array([x_component, y_component])

# get angle from components using atan2 version of arctangent function -- result is converted to degrees
def angle_from_vector(vector: np.ndarray) -> float:
    # use the arctangent function
    angle_in_radians = atan2(vector[1], vector[0])  
    # we are working in degrees -- python's functions return radians -- convert radians to degrees
    angle = angle_in_radians * 180.0 / pi
    # return the result
    return angle

# get length from components using Pythagorean theorem
def length_from_vector(vector: np.ndarray) -> float:
    length_squared = np.sum(vector**2)
    return sqrt(length_squared)

### Unit tests ###

class VectorTests(unittest.TestCase):
    
    def test_vector_from_length_and_angle(self):
        vector = vector_from_length_and_angle(1000.0, 30.0)
        expected_vector = np.array([866.02540378, 500.0])
        self.assertTrue(np.allclose(vector, expected_vector),
                        msg="Expected {}, but got {}".format(expected_vector, vector))
        
    def test_angle_from_vector(self):
        angle = angle_from_vector(np.array([866.02540378, 500.0]))
        self.assertAlmostEqual(angle, 30.0)
        
    def test_length_from_vector(self):
        length = length_from_vector(np.array([5.0, 12.0]))
        self.assertAlmostEqual(length, 13.0)

testSuite = unittest.TestLoader().loadTestsFromName("__main__.VectorTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)

## Functions to Get Acceleration of Gravity

All of these functions for gravity you implemented in Notebook 3-1. **Just execute this cell to get the gravity functions.**

In [ ]:
def strength_of_gravity(position):
    # this function encodes the strength of gravity as a function of distance from the center of the Earth
    radius = length_from_vector(position)
    strength = g * radius_of_earth**2 / radius**2
    return strength

def direction_of_gravity(position):
    # this function encodes the direction of gravity (the angle)
    # gravity is attractive -- it always points toward the center of the Earth
    direction = angle_from_vector(position) + 180.0
    return direction

def acceleration_of_gravity(position):
    # using the strength and direction functions you have just implemented compute and
    # return a 2x1 array for the acceleration of gravity
    strength = strength_of_gravity(position)
    direction = direction_of_gravity(position)
    acceleration = vector_from_length_and_angle(strength, direction)
    return acceleration

### Unit tests ###

class GravityTests(unittest.TestCase):

    # tests of strength

    def test_strength_of_gravity(self):
        position = np.array([radius_of_earth, 0.0])
        strength = strength_of_gravity(position)
        self.assertAlmostEqual(strength, g)

    def test_strength_of_gravity_high_up(self):
        position = np.array([0.0, 2.0 * radius_of_earth])
        strength = strength_of_gravity(position)
        self.assertAlmostEqual(strength, 0.25 * g)
        
    # tests of direction
        
    def test_direction_of_gravity_left(self):
        position = np.array([100.0, 0.0])
        direction = direction_of_gravity(position)
        self.assertEqual(direction, 180.0)

    def test_direction_of_gravity_up(self):
        position = np.array([0.0, -100.0])
        direction = direction_of_gravity(position)
        self.assertEqual(direction, 90.0)
        
    # tests of acceleration

    def test_acceleration_of_gravity_g_left(self):
        position = np.array([radius_of_earth, 0.0])
        acceleration = acceleration_of_gravity(position)
        expected_acceleration = np.array([-g, 0.0])
        self.assertTrue(np.allclose(acceleration, expected_acceleration),
                        msg="Expected {}, but got {}".format(expected_acceleration, acceleration))

    def test_acceleration_of_gravity_4g_up(self):
        position = np.array([0.0, -radius_of_earth / 2.0])
        acceleration = acceleration_of_gravity(position)
        expected_acceleration = np.array([0.0, 4.0 * g])
        self.assertTrue(np.allclose(acceleration, expected_acceleration),
                        msg="Expected {}, but got {}".format(expected_acceleration, acceleration))

testSuite = unittest.TestLoader().loadTestsFromName("__main__.GravityTests")
testRunner = unittest.TextTestRunner(verbosity=2)
testRunner.run(testSuite)


## The While Loop That Does the Work

Here is where your work lies.

You have learned about Kepler's laws for ellipses. Euler and Euler-Cromer have too much accumulated
error to work well with ellipses.

**Below you will be implementing second-order Runge-Kutta.** Look for the five TODO comments in the code. That's where your work is. I have tried to make second-order Runge-Kutta understandable.

In [ ]:
# Initialize the x and y velocities
velocities = np.zeros((simulation_time_steps, 2))
velocities[0] = initial_velocity

# Initialize the x and y positions
positions = np.zeros((simulation_time_steps, 2))
positions[0] = initial_position

# Initialize the times
times = np.zeros((simulation_time_steps,))

# The 0th elements of velocities, positions, and times are already filled in, so start with 1
for i in range(1, simulation_time_steps):
    #
    # get all the before values
    #
    # velocities
    before_velocity = velocities[i - 1]
    # positions
    before_position = positions[i - 1]
    # time
    before_time = times[i - 1]
    #
    # fundamental change for 2nd-order Runge-Kutta -- first estimate mid_position!!
    #
    mid_position = 0.0 # TODO
    #
    # calculate mid_acceleration -- using the estimated mid_position!
    # 
    mid_acceleration = 0.0 # TODO
    #
    # calculate after_velocity from before_velocity and mid_acceleration
    #
    after_velocity = 0.0 # TODO
    #
    # calculate after_position using the democratic combination of after_velocity and before_velocity
    #
    after_position = 0.0 # TODO
    #
    # update time
    #
    after_time = 0.0 # TODO
    #
    # assign the after values into their lists
    #
    velocities[i] = after_velocity
    positions[i] = after_position
    times[i] = after_time


## Diagram

**Execute this to make a graph that looks a little like [the diagram Newton made](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2f/Newton%27s_Principia_%281846%29.djvu/page519-1024px-Newton%27s_Principia_%281846%29.djvu.jpg).** Our ellipses are better than Newton's!

In [ ]:
plt.figure(figsize=(15, 15))

x_positions = positions[:, 0]
y_positions = positions[:, 1]
plt.scatter(x_positions, y_positions)

plt.xlabel("x position (km)")
plt.ylabel("y position (km)")

# Some gibberish that draws a big blue circle representing the Earth:
earth = plt.Circle((0, 0), radius_of_earth, color='b')
plt.gcf().gca().add_artist(earth)

# Make the plot big enough to show elliptical orbits:
plot_limit = 8000
plt.xlim(-1.8 * plot_limit, 1.8 * plot_limit)
plt.ylim(-2.4 * plot_limit, 1.2 * plot_limit)

plt.show()

## Playing with the Notebook and Checkout Questions

### Question 1a

Using the intial velocity of 8.982 km/s, where in the ellipse is the cannonball moving the slowest? How are you answering this?

### Question 1b

Where in the ellipse is the cannonball moving the fastest?

### Question 2a

Without changing the simulation duration, increase the initial velocity. Do you expect to get a larger or smaller ellipse? Is the ellipse more or less eccentric (more oval or more circular)?

### Question 2b

With the increased initial velocity, do you expect the orbit to over-close in the same simulation duration, or will it not close?

### Question 3a

Still without changing the simulation duration, decrease the initial velocity. Do you expect to get a larger or smaller ellipse? Is the ellipse more or less eccentric (more oval or more circular)?

### Question 3b

With the decreased initial velocity, do you expect the orbit to over-close in the same simulation duration, or will it not close?

### Question 4

What value of the initial velocity gets a circle?

## Conclusion

With oodles of software and hardware and just three weeks of time learning to use it, you can get more results than Isaac Newton.